In [6]:
import pandas as pd
import numpy as np 
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import random
from numpy import unravel_index
# For nice Plots
%matplotlib inline
plt.style.use('default')

import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras import models, layers, utils, backend as K
import matplotlib.pyplot as plt
import shap
df = pd.read_csv('data/BRFSS_clean.csv')

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
  df.iloc[:,3:], df['MENTHLTH'], test_size=0.20)

# Random Forest

## Initial Guess

In [12]:
random.seed(10)
rf= RandomForestClassifier(max_depth=10, random_state=0)
rf.fit(X_train, y_train)
#put our info in the random classifier
yhat_rf = rf.predict(X_test)
acc_rf = np.sum(yhat_rf==y_test)/len(y_test)
print("RF Accuracy: "+str(acc_rf))

RF Accuracy: 0.6261126457949663


In [13]:
mse = np.mean((yhat_rf - y_test)**2)
mse

72.59290592387968

## Cross Validation to Find Optimal Parameters

In [14]:
# Use cross validation to select a good value for k
acc_test = np.zeros((20, 49, 4))
acc_train = np.zeros((20, 49, 4))
acc_cv = np.zeros((20, 49, 4))
#make vectors to hold data, considering numbers for k from 1-50 
for k in range(1,21):
    for i in range(1,5):
        for j in range(1,50):
          rf = RandomForestClassifier(max_depth=k, n_estimators=j, max_features=i, random_state=0)
          acc_cv[k-1, j-1, i-1] = cross_val_score(rf, X_train, y_train, cv=10).mean()
          # Train 
          rf.fit(X_train, y_train)
          # Apply to Validatio/Test Data
          yhat_rf = rf.predict(X_test)
          acc_test[k-1, j-1, i-1] = np.sum(yhat_rf == y_test)/len(y_test)
          # Apply to Validatio/Test Data
          yhat_rf = rf.predict(X_train)
          acc_train[k-1, j-1, i-1] = np.sum(yhat_rf == y_train)/len(y_train)
#would expect accuracy on training data to be slightly better than on testing data

KeyboardInterrupt: 

In [15]:
unravel_index(acc_cv.argmax(), acc_cv.shape)
#13 max depth, 49 number of estimators, and 2 max features

(0, 0, 0)

# Neural Network